In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
import matplotlib.pyplot as plt

from lavian_et_al_2025.imaging.imaging_classes import LightsheetExperiment

import json
from pathlib import Path

from scipy import stats

## Set paths to the NTR and control datasets

In [ ]:
master_ntr = Path(r"")
master_control = Path(r"")

pre_list = list(master_ntr.glob("*_ntr_pre*"))
pre_list = [f for f in pre_list if (f / "data_from_suite2p_cells.h5").exists()]

post_list = list(master_ntr.glob("*_ntr_post*"))
post_list = [f for f in post_list if (f / "data_from_suite2p_cells.h5").exists()]


pre_list_ctrl = list(master_control.glob("*_control_pre*"))
pre_list_ctrl = [f for f in pre_list_ctrl if (f / "data_from_suite2p_cells.h5").exists()]

post_list_ctrl = list(master_control.glob("*_control_post*"))
post_list_ctrl = [f for f in post_list_ctrl if (f / "data_from_suite2p_cells.h5").exists()]

num_fish_ntr = len(pre_list)
num_fish_ctrl = len(pre_list)

In [ ]:
regions = ['ipn_coords', 'ahb_coords']
num_regions = len(regions)
thresh = 0.3
n_tuned_pre = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB
n_neurons_pre = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB
n_tuned_post = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB
n_neurons_post = np.zeros((num_regions, num_fish_ntr)) # IPN, aHB

## Find reliably responding ROIs in each region
#### NTR+ fish

In [ ]:
bins = np.linspace(-1, 1, 100)    
fig2, ax2 = plt.subplots(num_regions, num_fish_ntr, figsize=(12, 7))

for fish in range(num_fish_ntr):
    pre_fish = pre_list[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(pre_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_pre[region, fish] = num_traces
        n_tuned_pre[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='pre', density=False)
        ax2[region, 0].set_ylabel("Count(" + regions[region] + ")")
    
    ax2[0, fish].set_title(fish_id)
    ax2[0, fish].set_xlabel("Correlation")
    
    #######################################
    post_fish = post_list[fish]
    
    exp = Experiment(post_fish)
    fish_id = exp.fish_id
    print(fish_id)

    coords_regions = fl.load(post_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(post_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_post[region, fish] = num_traces
        n_tuned_post[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='post', density=False)
        ax2[0, num_fish_ntr-1].legend(loc='upper right')
        
        ax2[region, fish].spines['right'].set_visible(False)
        ax2[region, fish].spines['top'].set_visible(False)
   

In [ ]:
plt.subplots_adjust(left=0.05, right=0.95, wspace=0.2)

In [ ]:
for i in range(num_fish_ntr):
    ax2[0, i].set_xlim(-0.2,1)
    ax2[1, i].set_xlim(-0.2,1)

In [ ]:
n_tuned_pre_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB
n_neurons_pre_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB
n_tuned_post_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB
n_neurons_post_ctrl = np.zeros((num_regions, num_fish_ctrl)) # IPN, aHB

#### NTR- (control) fish

In [ ]:
bins = np.linspace(-1, 1, 100)    
fig2, ax2 = plt.subplots(num_regions, num_fish_ctrl, figsize=(12, 7))

for fish in range(num_fish_ctrl):
    pre_fish = pre_list_ctrl[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    coords_regions = fl.load(pre_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(pre_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_pre_ctrl[region, fish] = num_traces
        n_tuned_pre_ctrl[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='pre', density=False)
        ax2[region, 0].set_ylabel("Count(" + regions[region] + ")")
    
    ax2[0, fish].set_title(fish_id)
    ax2[0, fish].set_xlabel("Correlation")
    
    #######################################
    post_fish = post_list_ctrl[fish]
    
    exp = Experiment(post_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    coords_regions = fl.load(post_fish / "regions_masks_mapzbrain.h5")
    rel_ind_all = fl.load(post_fish / "reliability_index_arr.h5")['reliability_arr_combined']
    
    for region in range(num_regions):
        
        # getting region coords 
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        rel_ind = rel_ind_all[region_idx]

        n_neurons_post_ctrl[region, fish] = num_traces
        n_tuned_post_ctrl[region, fish] = len(np.where(rel_ind > thresh)[0])
        ax2[region, fish].hist(rel_ind, bins, alpha=0.5, label='post', density=False)
        ax2[0, num_fish_ctrl-1].legend(loc='upper right')
        
        ax2[region, fish].spines['right'].set_visible(False)
        ax2[region, fish].spines['top'].set_visible(False)
   

In [ ]:
num_corr_cells = np.append(n_tuned_pre, n_tuned_post, axis=0)
num_corr_cells_ipn = num_corr_cells[[0,2],:]
num_corr_cells_ahb = num_corr_cells[[1,3],:]

In [ ]:
fig1, ax1 = plt.subplots(2,2, figsize=(5,4))

ax1[0,0].plot(num_corr_cells_ipn, color='gray')
ax1[0,0].spines['right'].set_visible(False)
ax1[0,0].spines['top'].set_visible(False)
ax1[0,0].set_xlim(-0.5, 1.5)

ax1[0,1].plot(num_corr_cells_ahb, color='gray')
ax1[0,1].spines['right'].set_visible(False)
ax1[0,1].spines['top'].set_visible(False)
ax1[0,1].set_xlim(-0.5, 1.5)
ax1[0,0].set_ylabel('Cell count (IPN)')
ax1[0,1].set_ylabel('Cell count (aHB)')

fig1.subplots_adjust(left=0.15, right=0.95, wspace=0.6)
ax1[0,0].set_ylim(0, 100)
ax1[1,0].set_ylim(0, 100)
ax1[0,1].set_ylim(0, 900)
ax1[1,1].set_ylim(0, 900)

In [ ]:
ax1[0,0].errorbar([0, 1], np.nanmean(num_corr_cells_ipn, axis=1), np.nanstd(num_corr_cells_ipn, axis=1) / 11, color='k')
ax1[0,1].errorbar([0, 1], np.nanmean(num_corr_cells_ahb, axis=1), np.nanstd(num_corr_cells_ahb, axis=1) / 11, color='k')

In [ ]:
num_corr_cells_ctrl = np.append(n_tuned_pre_ctrl, n_tuned_post_ctrl, axis=0)
num_corr_cells_ipn_ctrl = num_corr_cells_ctrl[[0,2],:]
num_corr_cells_ahb_ctrl = num_corr_cells_ctrl[[1,3],:]

In [ ]:
ax1[1,0].plot(num_corr_cells_ipn_ctrl, color='gray')
ax1[1,0].spines['right'].set_visible(False)
ax1[1,0].spines['top'].set_visible(False)
ax1[1,0].set_xlim(-0.5, 1.5)

ax1[1,1].plot(num_corr_cells_ahb_ctrl, color='gray')
ax1[1,1].spines['right'].set_visible(False)
ax1[1,1].spines['top'].set_visible(False)
ax1[1,1].set_xlim(-0.5, 1.5)
ax1[1,0].set_ylabel('Cell count (IPN)')
ax1[1,1].set_ylabel('Cell count (aHB)')

In [ ]:
ax1[1,0].errorbar([0, 1], np.nanmean(num_corr_cells_ipn_ctrl, axis=1), np.nanstd(num_corr_cells_ipn_ctrl, axis=1) / 11, color='k')
ax1[1,1].errorbar([0, 1], np.nanmean(num_corr_cells_ahb_ctrl, axis=1), np.nanstd(num_corr_cells_ahb_ctrl, axis=1) / 11, color='k')

### Statistics

In [ ]:
############### t-test for ntr group
stats.ttest_rel(num_corr_cells_ipn[0], num_corr_cells_ipn[1], alternative='greater')

In [ ]:
############### t-test for ntr group
stats.ttest_rel(num_corr_cells_ahb[0], num_corr_cells_ahb[1], alternative='greater')

In [ ]:
############### t-test for control group
stats.ttest_rel(num_corr_cells_ipn_ctrl[0], num_corr_cells_ipn_ctrl[1], alternative='greater')

In [ ]:
############### t-test for control group
stats.ttest_rel(num_corr_cells_ahb_ctrl[0], num_corr_cells_ahb_ctrl[1], alternative='greater')